In [1]:
# import all libraries

from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 120)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('../data/train')
valid_files, valid_targets = load_dataset('../data/validation')
test_files, test_targets = load_dataset('../data/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("../data/train/*/"))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

There are 120 total dog categories.
There are 20580 total dog images.

There are 14531 training dog images.
There are 4232 validation dog images.
There are 1817 test dog images.


In [3]:
from keras.applications import ResNet50
from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D
from keras.models import Model, Sequential

pretrained_model= ResNet50(include_top=False, weights="imagenet", input_shape=(224, 224, 3)) 

for layer in pretrained_model.layers:
    layer.trainable = False
    
x = pretrained_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
#dropout
predictions = Dense(120, activation='softmax')(x)

model = Model(inputs = pretrained_model.input, outputs=predictions) 

base_model = ResNet50(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/home/bayet/.virtualenvs/py3cv4/lib/python3.5/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [4]:
# training image augmentation
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import optimizers
from keras.callbacks import History 
from keras.applications import vgg16

batch_size = 16

# this is the augmentation configuration I will use for training
train_datagen = ImageDataGenerator(rotation_range=20,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   preprocessing_function=vgg16.preprocess_input)

# This is the augmentation configuration I will use for testing/validation... just a rescale
test_datagen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input)

# This is the generator which will read pictures found in my training subset
train_generator = train_datagen.flow_from_directory('../data/train/',
                                                    target_size = (224, 224),
                                                    batch_size = batch_size,
                                                    shuffle=True,
                                                    class_mode = 'categorical',
                                                    seed=42)

# This is the generator for validation data
validation_generator = test_datagen.flow_from_directory('../data/validation/',
                                                        target_size = (224, 224),
                                                        batch_size = batch_size,
                                                        class_mode = 'categorical')

Found 14531 images belonging to 120 classes.
Found 4232 images belonging to 120 classes.


In [5]:
from keras.callbacks import ModelCheckpoint, History

history = History()
checkpoint_augmentation = '../data/checkpoint_augmentation/resnet'
model_checkpoint_augmentation = ModelCheckpoint(filepath=checkpoint_augmentation,
                                                verbose=1,
                                                save_best_only=True)

In [8]:
%%time
model.fit_generator(train_generator,
                    epochs = 10,
                    validation_data = validation_generator,
                    #callbacks = [model_checkpoint_augmentation, history],
                    verbose=2)

Epoch 1/10
 - 2127s - loss: 1.2266 - accuracy: 0.6378 - val_loss: 0.9512 - val_accuracy: 0.7006
Epoch 2/10
 - 2143s - loss: 1.0626 - accuracy: 0.6788 - val_loss: 0.4939 - val_accuracy: 0.6940
Epoch 3/10
 - 2066s - loss: 1.0005 - accuracy: 0.6966 - val_loss: 1.2360 - val_accuracy: 0.7098
Epoch 4/10
 - 2114s - loss: 0.9050 - accuracy: 0.7245 - val_loss: 1.5541 - val_accuracy: 0.6966
Epoch 5/10
 - 3246s - loss: 0.8651 - accuracy: 0.7379 - val_loss: 1.5739 - val_accuracy: 0.6928
Epoch 6/10
 - 2724s - loss: 0.7973 - accuracy: 0.7551 - val_loss: 1.4875 - val_accuracy: 0.7034
Epoch 7/10
 - 4366s - loss: 0.7530 - accuracy: 0.7638 - val_loss: 0.9672 - val_accuracy: 0.6997
Epoch 8/10
 - 4181s - loss: 0.7141 - accuracy: 0.7759 - val_loss: 0.3811 - val_accuracy: 0.7155
Epoch 9/10
 - 2153s - loss: 0.6715 - accuracy: 0.7921 - val_loss: 3.5684 - val_accuracy: 0.7101
Epoch 10/10
 - 2133s - loss: 0.6676 - accuracy: 0.7903 - val_loss: 3.4774 - val_accuracy: 0.6935
CPU times: user 1d 4h 55min 58s, sys: 7

In [8]:
1+1

2